# High performance pandas

use only when needed as worse readibility

```py
mask = (x > .5) & (y < .5)
df[df[mas]]
```

under the hood  
tmp1 = (x > .5)  
tmp2 = (y < .5)  
mask = tmp1 & tmp2

use pd.eval(""), pd.query-> does elemantwise operation with numexpr  
(python eval() is not save if used together with username)

In [1]:
import numpy as np
import pandas as pd

nrows, ncals = 1_000_000, 100

df1, df2, df3, df4 = [pd.DataFrame(np.random.randn(nrows, ncals)) for _ in range(4)]
df1.head()

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,-0.087099,-1.283371,-0.097058,-0.282993,0.176703,1.723934,-0.382770,-0.864502,-0.045458,1.761216,...,-1.242578,0.690452,1.202234,-0.152311,0.219373,0.394410,0.679052,-0.557621,-1.787483,0.387899
1,-0.569062,-1.995528,0.439232,-0.398954,-2.043263,-0.290632,-0.528904,-1.496011,-1.606071,-0.568234,...,0.714315,-0.961877,0.084177,-0.614458,-0.832605,-2.592647,-2.288741,0.975251,1.037554,0.515469
2,0.794712,0.669932,-1.033651,-0.466691,0.657023,0.067145,-2.931652,-0.631142,0.480100,-0.506580,...,0.800346,1.915103,0.052937,0.315153,-0.964624,-0.248293,-0.432277,1.426588,-0.654751,-0.461619
3,0.531262,1.047608,0.184973,-0.680984,1.052253,-0.237307,0.480801,-0.421134,-1.467633,-0.192245,...,1.468428,0.262986,0.910604,0.140509,-0.163262,-0.600724,0.711458,-2.100919,-2.087222,-1.675408
4,-0.803016,0.399209,0.867035,-1.271566,-0.680470,-0.609546,-1.010085,-0.407391,-1.380100,-0.783547,...,-0.480274,-0.326611,1.620810,0.304649,0.234617,-0.166992,-0.846853,-0.480422,-1.792000,-0.436894


In [2]:
df1.shape

(1000000, 100)

In [3]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 100 columns):
 #   Column  Non-Null Count    Dtype  
---  ------  --------------    -----  
 0   0       1000000 non-null  float64
 1   1       1000000 non-null  float64
 2   2       1000000 non-null  float64
 3   3       1000000 non-null  float64
 4   4       1000000 non-null  float64
 5   5       1000000 non-null  float64
 6   6       1000000 non-null  float64
 7   7       1000000 non-null  float64
 8   8       1000000 non-null  float64
 9   9       1000000 non-null  float64
 10  10      1000000 non-null  float64
 11  11      1000000 non-null  float64
 12  12      1000000 non-null  float64
 13  13      1000000 non-null  float64
 14  14      1000000 non-null  float64
 15  15      1000000 non-null  float64
 16  16      1000000 non-null  float64
 17  17      1000000 non-null  float64
 18  18      1000000 non-null  float64
 19  19      1000000 non-null  float64
 20  20      1000000 non-null

In [5]:
%timeit df1+df2+df3+df4

1.47 s ± 149 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [6]:
%timeit pd.eval("df1+df2+df3+df4")

517 ms ± 33 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [8]:
standard = df1+df2+df3+df4
sum_eval = pd.eval("df1+df2+df3+df4")

sum_eval.equals(standard)

True

In [11]:
rolls = pd.DataFrame(np.random.randint(1,6, (6,3)), columns = ["Die1", "Die2", "Die3"])
rolls

,Die1,Die2,Die3
0,1,1,4
1,2,3,4
2,1,4,3
3,3,1,1
4,1,2,3
5,1,2,5


In [16]:
rolls.eval("Sum = Die1 + Die2 + Die3", inplace = True)
rolls

,Die1,Die2,Die3,Sum
0,1,1,4,6
1,2,3,4,9
2,1,4,3,8
3,3,1,1,5
4,1,2,3,6
5,1,2,5,8


In [20]:
high  = 6
rolls.eval("Winner = Sum > @high")

,Die1,Die2,Die3,Sum,Winner
0,1,1,4,6,False
1,2,3,4,9,True
2,1,4,3,8,True
3,3,1,1,5,False
4,1,2,3,6,False
5,1,2,5,8,True


#### Filtering

In [21]:
# traditional filtering
rolls[rolls["Sum"] > high]

,Die1,Die2,Die3,Sum
1,2,3,4,9
2,1,4,3,8
5,1,2,5,8


In [26]:
# with query
rolls.query("Sum > @high & Die1 == 1")


,Die1,Die2,Die3,Sum
2,1,4,3,8
5,1,2,5,8


In [27]:
df_os = pd.read_csv("../Data/athlete_events.csv")
# https://raw.githubusercontent.com/DorotaBjoorn/Databehandling-Dorota-Bjoorn/main/Data/athlete_events.csv

In [29]:
df_os[df_os["NOC"] == "SWE"].head()

,ID,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal
725,414,Arvid berg,M,26.0,NaN,NaN,Sweden,SWE,1912 Summer,1912,Summer,Stockholm,Athletics,Athletics Men's Hammer Throw,NaN
726,415,Bjrn Olof Conny berg,M,23.0,181.0,76.0,Sweden,SWE,1992 Winter,1992,Winter,Albertville,Freestyle Skiing,Freestyle Skiing Men's Moguls,NaN
727,416,Nils Georg berg,M,19.0,181.0,78.0,Sweden,SWE,1912 Summer,1912,Summer,Stockholm,Athletics,Athletics Men's Long Jump,Bronze
728,416,Nils Georg berg,M,19.0,181.0,78.0,Sweden,SWE,1912 Summer,1912,Summer,Stockholm,Athletics,Athletics Men's Triple Jump,Silver
729,417,Sara Helena berg,F,17.0,190.0,73.0,Sweden,SWE,1988 Summer,1988,Summer,Seoul,Swimming,Swimming Women's 50 metres Freestyle,NaN


In [30]:
df_os.query("NOC == 'SWE'").head()
# 'SWE' since its a str value and not coulmn name as in Sum example or float as > 180 in example below

,ID,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal
725,414,Arvid berg,M,26.0,NaN,NaN,Sweden,SWE,1912 Summer,1912,Summer,Stockholm,Athletics,Athletics Men's Hammer Throw,NaN
726,415,Bjrn Olof Conny berg,M,23.0,181.0,76.0,Sweden,SWE,1992 Winter,1992,Winter,Albertville,Freestyle Skiing,Freestyle Skiing Men's Moguls,NaN
727,416,Nils Georg berg,M,19.0,181.0,78.0,Sweden,SWE,1912 Summer,1912,Summer,Stockholm,Athletics,Athletics Men's Long Jump,Bronze
728,416,Nils Georg berg,M,19.0,181.0,78.0,Sweden,SWE,1912 Summer,1912,Summer,Stockholm,Athletics,Athletics Men's Triple Jump,Silver
729,417,Sara Helena berg,F,17.0,190.0,73.0,Sweden,SWE,1988 Summer,1988,Summer,Seoul,Swimming,Swimming Women's 50 metres Freestyle,NaN


In [31]:
%timeit df_os[df_os["NOC"] == "SWE"]
%timeit df_os.query("NOC == 'SWE'")

23.3 ms ± 857 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
16.7 ms ± 1.76 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [33]:
df_os.query("Height > 180").head()

,ID,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal
4,5,Christine Jacoba Aaftink,F,21.0,185.0,82.0,Netherlands,NED,1988 Winter,1988,Winter,Calgary,Speed Skating,Speed Skating Women's 500 metres,NaN
5,5,Christine Jacoba Aaftink,F,21.0,185.0,82.0,Netherlands,NED,1988 Winter,1988,Winter,Calgary,Speed Skating,"Speed Skating Women's 1,000 metres",NaN
6,5,Christine Jacoba Aaftink,F,25.0,185.0,82.0,Netherlands,NED,1992 Winter,1992,Winter,Albertville,Speed Skating,Speed Skating Women's 500 metres,NaN
7,5,Christine Jacoba Aaftink,F,25.0,185.0,82.0,Netherlands,NED,1992 Winter,1992,Winter,Albertville,Speed Skating,"Speed Skating Women's 1,000 metres",NaN
8,5,Christine Jacoba Aaftink,F,27.0,185.0,82.0,Netherlands,NED,1994 Winter,1994,Winter,Lillehammer,Speed Skating,Speed Skating Women's 500 metres,NaN


In [34]:
%timeit df_os[df_os["Height"] > 180]
%timeit df_os.query("Height > 180") # in this example slower because only single question/filter

16.4 ms ± 475 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
22.5 ms ± 1.04 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [37]:
df_os[(df_os["Sex"] == "F") & (df_os["Height"] > 180) & (df_os["NOC"] == 'SWE')].head(2)

,ID,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal
729,417,Sara Helena berg,F,17.0,190.0,73.0,Sweden,SWE,1988 Summer,1988,Summer,Seoul,Swimming,Swimming Women's 50 metres Freestyle,NaN
5175,2940,Jenny Alm,F,27.0,184.0,80.0,Sweden,SWE,2016 Summer,2016,Summer,Rio de Janeiro,Handball,Handball Women's Handball,NaN


In [38]:
df_os.query("Sex == 'F' & Height > 180 & NOC == 'SWE'").head(2)

,ID,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal
729,417,Sara Helena berg,F,17.0,190.0,73.0,Sweden,SWE,1988 Summer,1988,Summer,Seoul,Swimming,Swimming Women's 50 metres Freestyle,NaN
5175,2940,Jenny Alm,F,27.0,184.0,80.0,Sweden,SWE,2016 Summer,2016,Summer,Rio de Janeiro,Handball,Handball Women's Handball,NaN


In [39]:
%timeit df_os[(df_os["Sex"] == "F") & (df_os["Height"] > 180) & (df_os["NOC"] == "SWE")]
%timeit df_os.query("Sex == 'F' & Height > 180 & NOC == 'SWE'") #faster for multiple filter statements

42.3 ms ± 1.23 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
26.4 ms ± 1.36 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
